In [36]:
from langchain.agents import Tool, LLMSingleActionAgent, AgentExecutor
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os

# 1. 准确定义工具（名称和描述要清晰）
def get_word_length(word: str) -> int:
    return len(word)

tools = [
    Tool(
        name="字符串长度计算",  # 工具名称必须唯一且明确
        func=get_word_length,
        description="计算英文单词的长度，输入一个单词，返回它的字符数"
    )
]


In [37]:
# 查看工具信息
print(tools[0].name)  
print(tools[0].func) 
print(tools[0].description) 

print(get_word_length('hello'))

字符串长度计算
<function get_word_length at 0x000001BC76C95300>
计算英文单词的长度，输入一个单词，返回它的字符数
5


In [42]:
# 2. 重构Prompt模板（强制要求Final Answer）
prompt = PromptTemplate.from_template("""
请严格按以下格式响应：

Question: {input}

Thought: 分析问题
Action: 工具名称
Action Input: "输入内容"
Observation: 工具返回结果
...（重复如果需要）
Final Answer: 最终答案

必须使用工具:
{tools}

不允许使用其他工具。

现在开始：

Question: {input}
""")

# 创建llm
# BASE_URL="https://dashscope.aliyuncs.com/compatible-mode/v1"
# MODEL = "qwen-turbo"
# API_KEY = os.getenv('DASHSCOPE_API_KEY')
# llm = ChatOpenAI(
#     model= MODEL,  # 默认的大模型为GPT-3.5-turbo，比较便宜
#     openai_api_base= BASE_URL,
#     openai_api_key= API_KEY
# )
API_KEY="sk-kEjwwIwe4w4EhkfGei4kNkawKIZeyEfezSdLfiyJrR4jdgPl"
BASE_URL = "https://api.fe8.cn/v1"
MODEL= "gpt-4o-mini"
llm = ChatOpenAI(
    model= MODEL,  # 默认的大模型为GPT-3.5-turbo，比较便宜
    openai_api_base= BASE_URL,
    openai_api_key= API_KEY
)

# Create LLMChain first
llm_chain = LLMChain(llm=llm, prompt=prompt)
# llm_chain =llm|prompt
# Create output parser
from langchain.agents.output_parsers import ReActSingleInputOutputParser
output_parser = ReActSingleInputOutputParser()
# output_parser=JSONAgentOutputParser()
# 定义的工具名称
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    allowed_tools=tool_names,  # optional but recommended
    stop=["\nObservation:", "\nFinal Answer:"]  # 双保险停止条件
)

# 5. 执行时传入工具描述
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=5  # 防止无限循环
)
# 6. 运行（确保输入格式正确）
result = agent_executor.invoke({
    "input": "单词'hello'的长度是多少？",
    "tools": "\n".join([f"{tool.name}: {tool.description}" for tool in tools])
})

print(result["output"])  # 输出: 5



> Entering new AgentExecutor chain...
Thought: 分析问题
Action: 字符串长度计算
Action Input: "hello"


Observation:7
Question: 单词'hello'的长度是多少？

Thought: 分析问题
Action: 字符串长度计算
Action Input: "hello"


Observation:7
Question: 单词'hello'的长度是多少？

Thought: 分析问题
Action: 字符串长度计算
Action Input: "hello"


Observation:7
Question: 单词'hello'的长度是多少？

Thought: 分析问题
Action: 字符串长度计算
Action Input: "hello"


Observation:7
Question: 单词'hello'的长度是多少？

Thought: 分析问题
Action: 字符串长度计算
Action Input: "hello"


Observation:7


> Finished chain.
Agent stopped due to iteration limit or time limit.
